## Consolida fatores de risco em uma só dataframe, e elimina cols. em duplicidade

## Monica Barros - 26/02/2021

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('fivethirtyeight')
pd.set_option('max_colwidth', 200)  # Altera largura máxima das colunas
pd.set_option('display.max_colwidth', None)

import seaborn as sns; 
sns.set(rc={'figure.figsize':(17,6)})

plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

import os
import datetime
from dateutil.relativedelta import relativedelta
import time
from time import perf_counter

import glob
import pickle

#!pip install openpyxl
from openpyxl import Workbook

#!pip install pytz
import pytz
local_tz = pytz.timezone('America/Sao_Paulo')

In [2]:
dir_cadastro =  '/home/jovyan/work/@Fund_Eval/CVM/Cadastro/'
dir_cot_fundos = '/home/jovyan/work/@Fund_Eval/CVM/Diario/'
dir_fat_risco = '/home/jovyan/work/@Fund_Eval/@fatores_risco/'

In [3]:
os.chdir(dir_fat_risco)

In [4]:
os.listdir()

['df_adj_close_all_2021_02_18_2318.csv',
 'df_close_all_2021_02_18_2318.csv',
 'fat_risco_2021_02_19_2216.csv',
 'fat_risco_2021_02_19_2216.pkl',
 'fat_risco_2021_02_25_2030.csv',
 'fat_risco_2021_02_25_2030.pkl',
 'fatores_de_risco_ready_to_go_20210223.csv',
 'fatores_de_risco_ready_to_go_20210223.pkl',
 'fundos_IHFA.csv',
 'fundos_mm_ready_to_go_20210223.pkl',
 'indices_internacionais_2021_02_23_2143.csv',
 'indices_internacionais_2021_02_23_2143.pkl',
 'indices_internacionais_2021_02_25_2025.csv',
 'indices_internacionais_2021_02_25_2025.pkl',
 'indices_internacionais_2021_02_26_1331.csv',
 'indices_internacionais_2021_02_26_1331.pkl',
 'small_fundos_mm_ready_to_go_20210223.csv',
 'small_fundos_mm_ready_to_go_20210223.pkl']

### Define data de hoje (para incluir nos arquivos de saida)

In [29]:
data_hoje = datetime.datetime.now(tz = local_tz)
# Converte para string (incluindo hora e minuto - útil se for rodar várias vezes no mesmo dia para testes)
data_hoje = data_hoje.strftime("%Y_%m_%d_%H%M")

dd = datetime.datetime.now(tz = local_tz).strftime('%Y%m%d')

In [5]:
#df1 = pd.read_pickle('indices_internacionais_2021_02_23_2143.pkl')
df1 = pd.read_pickle('indices_internacionais_2021_02_26_1331.pkl')

In [6]:
df1.columns.tolist()

['SP500',
 'NASDAQ',
 'Dow_Jones_Comp_Avg',
 'Dow_Jones_Ind_Avg',
 'Dow_Jones_Transp_Avg',
 'Wilshire5000',
 'VIX',
 'OIL_Brent',
 'OIL_WTI',
 '10yr_Treasury',
 '5yr_Treasury',
 '1yr_Treasury',
 '3mo_Treasury']

In [16]:
#df2 = pd.read_pickle('fat_risco_2021_02_19_2216.pkl')  
df2 = pd.read_pickle('fat_risco_2021_02_25_2030.pkl')

In [17]:
df2.columns.tolist()

['YEAR',
 'MONTH',
 'ANBIMA366_TJTLN1366',
 'ANBIMA366_TJTLN3366',
 'ANBIMA366_TJTLN6366',
 'ANBIMA366_TJTLN12366',
 'BMF366_FUT1DOL366',
 'BMF366_FUT3DOL366',
 'BMF366_FUT3DOLV366',
 'GM366_ERTUR366',
 'GM366_ERV366',
 'GM366_ERVV366',
 'GM366_EUROV366',
 'EIA366_PBRENT366',
 'EIA366_PWTI366',
 'GM366_DOW366',
 'SGS366_NASDAQ366',
 'VALOR366_FEDFUND366',
 'GM366_IBVSP366',
 'GM366_IBVSPV366',
 'IRF-M',
 'IMA-S',
 'IMA-C',
 'IMA-B',
 'IMA-B5',
 'IMA-B5plus',
 'IMA-Geral',
 'IRF-M1',
 'IRF-M1plus',
 'IMA-Geral_exC']

In [18]:
print(df1.shape)
print(df2.shape)

(1084, 13)
(1514, 30)


In [19]:
df_all=pd.concat([df1,df2], axis = 1)

In [20]:
print(df_all.shape)

(1514, 43)


In [63]:
#df_all.info()

In [21]:
cols_reorder = ['YEAR',
 'MONTH',
 'SP500',
 'NASDAQ',
 'SGS366_NASDAQ366',
 'GM366_DOW366',
 'Dow_Jones_Comp_Avg',
 'Dow_Jones_Ind_Avg',
 'Dow_Jones_Transp_Avg',
 'Wilshire5000',
 'VIX',
 'OIL_Brent',
 'EIA366_PBRENT366',
 'OIL_WTI',
 'EIA366_PWTI366',
 '10yr_Treasury', '5yr_Treasury',
 '1yr_Treasury',
 '3mo_Treasury',
 'VALOR366_FEDFUND366',
 'ANBIMA366_TJTLN1366',
 'ANBIMA366_TJTLN3366',
 'ANBIMA366_TJTLN6366',
 'ANBIMA366_TJTLN12366',
 'BMF366_FUT1DOL366',
 'BMF366_FUT3DOL366',
 'BMF366_FUT3DOLV366',
 'GM366_ERTUR366',
 'GM366_ERV366',
 'GM366_ERVV366',
 'GM366_EUROV366',
 'GM366_IBVSP366',
 'GM366_IBVSPV366',
 'IRF-M',
 'IMA-S',
 'IMA-C',
 'IMA-B',
 'IMA-B5',
 'IMA-B5plus',
 'IMA-Geral',
 'IRF-M1',
 'IRF-M1plus',
 'IMA-Geral_exC']

In [22]:
df_all = df_all[cols_reorder]

In [23]:
df_all.iloc[:,0:20].head()

,YEAR,MONTH,SP500,NASDAQ,SGS366_NASDAQ366,GM366_DOW366,Dow_Jones_Comp_Avg,Dow_Jones_Ind_Avg,Dow_Jones_Transp_Avg,Wilshire5000,VIX,OIL_Brent,EIA366_PBRENT366,OIL_WTI,EIA366_PWTI366,10yr_Treasury,5yr_Treasury,1yr_Treasury,3mo_Treasury,VALOR366_FEDFUND366
Date,,,,,,,,,,,,,,,,,,,,
2017-01-01,2017,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.625
2017-01-02,2017,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.625
2017-01-03,2017,1,2257.83,5429.08,5429.0,19882.0,6917.33,19881.76,9023.86,23465.37,12.85,55.05,55.05,52.36,52.36,2.45,1.94,0.89,0.53,0.625
2017-01-04,2017,1,2270.75,5477.00,5477.0,19942.0,6952.31,19942.16,9115.72,23639.16,11.85,54.57,54.57,53.26,53.26,2.46,1.94,0.87,0.53,0.625
2017-01-05,2017,1,2269.00,5487.94,5488.0,19899.0,6931.03,19899.29,9051.76,23598.61,11.67,54.99,54.99,53.77,53.77,2.37,1.86,0.83,0.52,0.625


In [24]:
df_all.iloc[:,0:20].tail()

,YEAR,MONTH,SP500,NASDAQ,SGS366_NASDAQ366,GM366_DOW366,Dow_Jones_Comp_Avg,Dow_Jones_Ind_Avg,Dow_Jones_Transp_Avg,Wilshire5000,VIX,OIL_Brent,EIA366_PBRENT366,OIL_WTI,EIA366_PWTI366,10yr_Treasury,5yr_Treasury,1yr_Treasury,3mo_Treasury,VALOR366_FEDFUND366
Date,,,,,,,,,,,,,,,,,,,,
2021-02-21,2021,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125
2021-02-22,2021,2,3876.50,13533.05,13533.05,31521.69,10416.13,31521.69,13381.42,40801.02,23.45,64.73,NaN,61.67,NaN,1.37,0.61,0.06,0.03,0.125
2021-02-23,2021,2,3881.37,13465.20,13465.20,31537.35,10432.98,31537.35,13403.11,40799.03,23.11,NaN,NaN,NaN,NaN,1.37,0.59,0.08,0.04,NaN
2021-02-24,2021,2,3925.43,13597.97,13597.97,31961.86,10547.40,31961.86,13630.55,41279.04,21.34,NaN,NaN,NaN,NaN,1.38,0.62,0.08,0.03,NaN
2021-02-25,2021,2,3829.34,13119.43,NaN,NaN,10363.89,31402.01,13372.26,40196.86,28.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
## Excluir colunas repetidas e Fed Funds (varia muito pouco)
df_all.drop(['SGS366_NASDAQ366', 'GM366_DOW366', 'EIA366_PBRENT366','EIA366_PWTI366', 'VALOR366_FEDFUND366'], axis=1, inplace=True)

In [26]:
df_all.shape

(1514, 38)

## Salva em pkl e csv

In [31]:
time_start = perf_counter()

df_all.to_pickle(dir_fat_risco + 'fatores_de_risco_ready_to_go_'+dd+'.pkl') 
df_all.to_csv(dir_fat_risco + 'fatores_de_risco_ready_to_go_'+dd+'.csv',sep = ';' ,encoding='utf-8') 

time_end = perf_counter()
elapsed_time = time_end - time_start
print(' ****salvar TODOS FATORES DE RISCO demorou', round(elapsed_time,2) , ' segundos *****')

 ****salvar TODOS FATORES DE RISCO demorou 0.35  segundos *****
